In [107]:
import pandas as pd
import xgboost as xgb

In [97]:
train_raw = pd.read_json('../contracts/artifacts/train.json')
# train
# 0 - addr
# 1 - tokens
# 2 - days
# 3 - returned


In [101]:
train = train_raw.groupby(0)[[1,2]].sum()
train = train.rename(columns={1: 'loans_tokens_sum', 2: 'loans_days_sum'})
train.index.rename('id', inplace=True)

In [102]:
train_y = train_raw.groupby(0)[[1]].sum()
train_y = train_y.rename(columns={1: 'trust'})
train_y.index.rename('id', inplace=True)

success = train_raw[train_raw[3]==True].groupby(0)[[1]].sum()
success = success.rename(columns={1: 'success'})
success.index.rename('id', inplace=True)
train_y['trust'] = (success['success'].fillna(0) / train_y['trust']).fillna(0)



In [111]:
train['loans_count'] = train_raw.groupby(0)[1].count()
train['success_loans_count'] = train_raw[train_raw[3]==True].groupby(0)[1].count()
train['avg_tokens'] = train['loans_tokens_sum'] / train['loans_count']
train['avg_days'] = train['loans_days_sum'] / train['loans_count']

train


,loans_tokens_sum,loans_days_sum,loans_count,success_loans_count,avg_tokens,avg_days
id,,,,,,
0x001af749a76f3d536865186391135fb74d343068,4455,53,18,11.0,247.500000,2.944444
0x00d7203cee608ae3eeb7dad6836e3fccdc8a6e74,14750,43,11,NaN,1340.909091,3.909091
0x010b4d633bbf7c6d88d737075583a62582146d48,24068,33,33,26.0,729.333333,1.000000
0x02ccae6707367c20af1186dd34ef711f73947c28,14525,18,13,NaN,1117.307692,1.384615
0x04c876e747eae8937e69157d90ba50db2506bc51,7612,13,13,11.0,585.538462,1.000000
0x0515263733fe4a605e0c6d8c7a1ea7f02a992ef1,7063,16,8,NaN,882.875000,2.000000
0x07b601f4dbd6dfb1eeb4b40bde3d177964f520e9,8140,23,23,22.0,353.913043,1.000000
0x07e8e64d4787c885fa74205d81247115c43cc1c8,17472,37,33,16.0,529.454545,1.121212
0x0814101b4251d9cb68240ac8c67658adf580634e,8260,53,37,25.0,223.243243,1.432432


In [108]:
trainDmatrix = xgb.DMatrix(train, label=train_y['trust'])

In [109]:
booster = xgb.train({}, trainDmatrix)
#booster.predict(xgb.DMatrix(train))

In [112]:
booster.save_model('model.bin')

In [114]:
bst = xgb.Booster() 
bst.load_model('model.bin')
bst.predict(xgb.DMatrix(train))

array([0.6668166 , 0.01578236, 0.7729696 , 0.01578236, 0.8902302 ,
       0.01578236, 0.944914  , 0.5195162 , 0.7337466 , 0.01578236,
       0.64919376, 0.95508766, 0.9260404 , 0.9840386 , 0.01578236,
       0.04530805, 0.01578236, 0.03470847, 0.9731995 , 0.02111775,
       0.9702951 , 0.9823013 , 0.03214112, 0.01578236, 0.63204634,
       0.01578236, 0.971943  , 0.98220396, 0.552181  , 0.9702951 ,
       0.17461577, 0.98226476, 0.9407058 , 0.01578236, 0.01578236,
       0.12530518, 0.3306674 , 0.9804231 , 0.72722936, 0.99134433,
       0.2969223 , 0.24025258, 0.98168206, 0.98899144, 0.99134433,
       0.46923023, 0.01578236, 0.5195162 , 0.01899537, 0.01578236,
       0.9902647 , 0.02111775, 0.9865915 , 0.09949449, 0.33233505,
       0.97495604, 0.9882903 , 0.01578236, 0.01578236, 0.7217306 ,
       0.14245766, 0.9804231 , 0.01578236, 0.07222545, 0.88456476,
       0.97621495, 0.9858705 , 0.01578236, 0.01578236, 0.72722936,
       0.02111775, 0.9790871 , 0.7051244 , 0.9597719 , 0.26178